[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/modelagem_4.ipynb)

# Modelagem e Gestão de Processos


**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**



In [64]:
import simpy # usado para a modelagem
import random # gerador de números randômicos
import numpy as np # usado para calcular algumas métricas
import pandas as pd # usado para limpeza de dados e DataFrames 

In [65]:
random.seed(2)

### Exemplo de processo que não alcança estado estável

Exemplo de processo M/M/2, com $\lambda = 1$ cliente/minuto e $\mu = 0,2$ clientes/minuto.

Como $\rho = \frac{\lambda}{s\times \mu} = \frac{1}{0,4} >1$, é esperado que o processo não estabilize.

A fila explodirá. 

Altere o código e calcule o tempo médio de espera dos clientes que finalizaram o serviço.

In [66]:
def estabelecimento (environment, nome_cliente, servidor):
    #imprima na tela o tempo de chegada.
    print (f"{nome_cliente} chega ao estabelecimento em {environment.now}") 
    
    #guarde a informação na variável tempo_chegada.
    tempo_chegada = environment.now 
    
    # A atividade requer um servidor. Usamos o with para demonstrar isso. 
    with servidor.request() as req:
        # Espere até o servidor estar disponível e guarde o tempo.
        yield req 
        
        # Escreva na tela o tempo de início do serviço
        print (f"{nome_cliente} inicia o serviço em {environment.now}") 
        # guarde o tempo de início em uma variável chamada tempo_inicio
        tempo_inicio = environment.now

        # tempo de espera será o tempo_inicio - tempo_chegada 
        tempo_em_fila.append(tempo_inicio - tempo_chegada)

        # O tempo estimado segue uma exponencial trabalha com Mu=0,2
        yield environment.timeout(random.expovariate(0.2))  
        
        #Imprima na tela o tempo de saída do sistema
        print (f"{nome_cliente} finaliza o serviço e sai do estabelecimento em {environment.now}")
        tempo_saida = environment.now
        
        # O tempo no sistema será tempo_saida - tempo_chegada
        tempo_no_sistema.append(tempo_saida - tempo_chegada) 

def chegadas (environment):
    id = 1 #guarda o id do cliente 
    
    # Enquanto houver simulação:
    while True:
        # Passa um tempo até a próxima chegada, seguindo exponencial com média (1/lambda)
        yield environment.timeout(random.expovariate(1))
        
        # Um cliente chega no processo
        environment.process(estabelecimento (environment, 'Cliente %d' % id, servidor))
        
        # O próximo cliente terá id = id + 1
        id += 1



In [67]:
tempo_em_fila = [] # uma lista vazia que receberá os tempos em fila
tempo_no_sistema = [] # uma lista vazia que receberá os tempos no sistema de filas

ambiente = simpy.Environment()
servidor = simpy.Resource(ambiente, capacity=2)
ambiente.process(chegadas(ambiente))
ambiente.run(until = 480)

Cliente 1 chega ao estabelecimento em 3.124344855143063
Cliente 1 inicia o serviço em 3.124344855143063
Cliente 1 finaliza o serviço e sai do estabelecimento em 3.4154116524293063
Cliente 2 chega ao estabelecimento em 6.077544349942528
Cliente 2 inicia o serviço em 6.077544349942528
Cliente 3 chega ao estabelecimento em 6.166235677432498
Cliente 3 inicia o serviço em 6.166235677432498
Cliente 4 chega ao estabelecimento em 7.497928181958402
Cliente 5 chega ao estabelecimento em 7.8662947178158715
Cliente 6 chega ao estabelecimento em 8.797557386064213
Cliente 7 chega ao estabelecimento em 9.730998685839106
Cliente 8 chega ao estabelecimento em 10.601370077828847
Cliente 9 chega ao estabelecimento em 10.773800161249985
Cliente 10 chega ao estabelecimento em 11.33709457755939
Cliente 3 finaliza o serviço e sai do estabelecimento em 11.705465641049894
Cliente 4 inicia o serviço em 11.705465641049894
Cliente 11 chega ao estabelecimento em 11.837197594835292
Cliente 2 finaliza o serviço e sa

In [68]:
print (f"Número de clientes que iniciaram o atendimento : {len(tempo_em_fila)}")
print (f"O tempo de espera médio: {np.mean(tempo_em_fila).round(2)} minutos")
print (f"O tempo médio no sistema: {np.mean(tempo_no_sistema).round(2)} minutos")

Número de clientes que iniciaram o atendimento : 193
O tempo de espera médio: 145.09 minutos
O tempo médio no sistema: 148.6 minutos


Veja que o tempo de espera médio foi de mais de 2 horas.

Na verdade, só contabilizamos tempos de espera que conseguiram iniciar o atendimento.

Perceba que chegaram mais de 400 clientes e menos da metade deles foram atendidos.

Seria interessante termos uma forma de guardar a utilização dos recursos e o número de clientes em fila!

# Analisando Recursos

A análise de recursos não é tão simples quanto guardar os tempos dos eventos. Para realizar uma análise efetiva, precisamos acessar os recursos após cada mudança no sistema.

A boa notícia é que já há uma função pronta disponibilizada na documentação do SimPy, copiada na célula abaixo.

Nesse código, usaremos decoradores de função (wrapper) para modificar recursos do SimPy, de modo que armazenaremos algumas características  antes e após alguma no mesmo.

O código abaixo é semelhante ao disponível na documentação. Algumas modificações foram realizadas em termos usados e nos comentários.

- https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html

In [69]:
from functools import partial, wraps

def modifica_recurso (recurso, pre=None, post=None):
    '''
    Modificaremos o recurso para que este chame o *pre* antes de cada operação 
    e o *post* após cada operação.
    *pre* e *post* fazem parte do wraps, que chamamos do pacote functools.
    Operações típicas de recursos do SimPy são: put, get, request e release.
    '''

    def get_decorador(func):
        
        # Cria um decorador para as funções internas dos recursos: put/get/request/release
        @wraps(func)
        def decorador(*args, **kwargs):
            # Chame o "pre" antes de aplicar a função
            if pre:
                pre(recurso)

            # aplique a função
            ret = func(*args, **kwargs)
            
            # Chame o "post" após aplicar a função
            if post:
                post(recurso)
            
            return ret #retorna o que a função retornaria
        return decorador # retorna a função modificada pelo decorador

    # Modifique a operação original pela modificada com o wrapper
    for nome in ['put', 'get', 'request', 'release']:
        
        # Se acontecer uma operação do tipo nome no recurso em questão, mude os atributos
        if hasattr(recurso, nome):
            setattr(recurso, nome, get_decorador(getattr(recurso, nome)))


# Armazenaremos o que queremos do recurso em questão em uma lista chamada dados
def monitoramento(dados_controle, recurso):
    item = (
            recurso._env.now,  # O tempo de simulação na ocorrência de uma operação
            recurso.count,  # Número de recursos do tipo em questão sendo utilizados
            len(recurso.queue),  # Número de trabalhos em fila para o recurso
    )   
    dados_controle.append(item)

In [70]:
tempo_em_fila = [] # uma lista vazia que receberá os tempos em fila
tempo_no_sistema = [] # uma lista vazia que receberá os tempos no sistema de filas

ambiente2 = simpy.Environment()
servidor = simpy.Resource(ambiente2, capacity=2)

dados_controle=[]
monitoramento = partial(monitoramento, dados_controle)
modifica_recurso (servidor, post = monitoramento)


ambiente2.process(chegadas(ambiente2))
ambiente2.run(until = 480)

Cliente 1 chega ao estabelecimento em 0.5436104524730686
Cliente 1 inicia o serviço em 0.5436104524730686
Cliente 1 finaliza o serviço e sai do estabelecimento em 0.6357342960807838
Cliente 2 chega ao estabelecimento em 2.33631942930974
Cliente 2 inicia o serviço em 2.33631942930974
Cliente 2 finaliza o serviço e sai do estabelecimento em 3.0432979343328337
Cliente 3 chega ao estabelecimento em 3.2033358678931494
Cliente 3 inicia o serviço em 3.2033358678931494
Cliente 4 chega ao estabelecimento em 3.3664804295241084
Cliente 4 inicia o serviço em 3.3664804295241084
Cliente 4 finaliza o serviço e sai do estabelecimento em 3.701732539306674
Cliente 5 chega ao estabelecimento em 3.8376072202298235
Cliente 5 inicia o serviço em 3.8376072202298235
Cliente 6 chega ao estabelecimento em 4.7248037143494575
Cliente 7 chega ao estabelecimento em 5.760746273424509
Cliente 8 chega ao estabelecimento em 6.444988079770921
Cliente 3 finaliza o serviço e sai do estabelecimento em 7.87141974167509
Clie

## Vejamos os dados que conseguimos!

In [71]:
dados_controle

[(0.5436104524730686, 1, 0),
 (0.6357342960807838, 0, 0),
 (2.33631942930974, 1, 0),
 (3.0432979343328337, 0, 0),
 (3.2033358678931494, 1, 0),
 (3.3664804295241084, 2, 0),
 (3.701732539306674, 1, 0),
 (3.8376072202298235, 2, 0),
 (4.7248037143494575, 2, 1),
 (5.760746273424509, 2, 2),
 (6.444988079770921, 2, 3),
 (7.87141974167509, 1, 3),
 (8.058370139857583, 2, 3),
 (8.222385865570663, 2, 4),
 (8.578082446406665, 2, 5),
 (11.924461264506593, 2, 6),
 (14.527641829475154, 2, 7),
 (14.75443368546865, 2, 8),
 (15.970621921782122, 2, 9),
 (16.12624142143231, 1, 9),
 (18.044529097448443, 2, 9),
 (19.255953023040068, 2, 10),
 (19.998255871038545, 2, 11),
 (20.248854599261783, 1, 11),
 (20.26571820539536, 2, 11),
 (20.329611259720846, 2, 12),
 (20.60039043209204, 1, 12),
 (21.344956599015326, 1, 11),
 (21.425155251685332, 2, 11),
 (21.446863123289532, 1, 11),
 (21.921483723540046, 2, 11),
 (24.311397954496485, 1, 11),
 (25.442996506631964, 2, 11),
 (26.086902845464934, 2, 12),
 (26.2020117218

### Organizando os dados dataframe

In [72]:
servidor_dados = pd.DataFrame(dados_controle, columns = ["Tempo", "n", "n_q"])
servidor_dados

,Tempo,n,n_q
0,0.543610,1,0
1,0.635734,0,0
2,2.336319,1,0
3,3.043298,0,0
4,3.203336,1,0
...,...,...,...
658,477.066143,1,330
659,477.082796,2,330
660,477.815569,2,331
661,478.585807,2,332


### Podemos adicionar uma linha no topo para ser o estado inicial, quando nenhum cliente chegou:

In [73]:
servidor_dados.loc[-1] = [0.0, 0, 0]  # adding a row
servidor_dados.index = servidor_dados.index + 1  # shifting index
servidor_dados.sort_index(inplace=True) 
servidor_dados

,Tempo,n,n_q
0,0.000000,0.0,0.0
1,0.543610,1.0,0.0
2,0.635734,0.0,0.0
3,2.336319,1.0,0.0
4,3.043298,0.0,0.0
...,...,...,...
659,477.066143,1.0,330.0
660,477.082796,2.0,330.0
661,477.815569,2.0,331.0
662,478.585807,2.0,332.0


### Agora adicionarems uma linha no fim com o estado do sistema no momento 480.

In [74]:
servidor_dados.loc[len(servidor_dados)] = [480, 2, 333]
servidor_dados

,Tempo,n,n_q
0,0.000000,0.0,0.0
1,0.543610,1.0,0.0
2,0.635734,0.0,0.0
3,2.336319,1.0,0.0
4,3.043298,0.0,0.0
...,...,...,...
660,477.082796,2.0,330.0
661,477.815569,2.0,331.0
662,478.585807,2.0,332.0
663,478.897545,2.0,333.0


### Queremos o tempo em que observamos cada estado antes de um novo evento!

Podemos chegar a isso com "servidor_1.diff()" aplicado à coluna tempo

In [75]:
servidor_dados["Intervalo"] = servidor_dados["Tempo"].diff().shift(-1)
servidor_dados

,Tempo,n,n_q,Intervalo
0,0.000000,0.0,0.0,0.543610
1,0.543610,1.0,0.0,0.092124
2,0.635734,0.0,0.0,1.700585
3,2.336319,1.0,0.0,0.706979
4,3.043298,0.0,0.0,0.160038
...,...,...,...,...
660,477.082796,2.0,330.0,0.732772
661,477.815569,2.0,331.0,0.770239
662,478.585807,2.0,332.0,0.311738
663,478.897545,2.0,333.0,1.102455


A última linha ficou com um NaN. Vamos excluí-la:


In [76]:
servidor_dados = servidor_dados.dropna(axis = 0)
servidor_dados

,Tempo,n,n_q,Intervalo
0,0.000000,0.0,0.0,0.543610
1,0.543610,1.0,0.0,0.092124
2,0.635734,0.0,0.0,1.700585
3,2.336319,1.0,0.0,0.706979
4,3.043298,0.0,0.0,0.160038
...,...,...,...,...
659,477.066143,1.0,330.0,0.016653
660,477.082796,2.0,330.0,0.732772
661,477.815569,2.0,331.0,0.770239
662,478.585807,2.0,332.0,0.311738


## Calculando métricas interessantes

### Número médio de clientes em fila


Sabemos que a fila está explodindo. Vejamos a média de clientes em fila nesses 480 min simulados


In [77]:
servidor_dados["Intervalo"] @ servidor_dados["n_q"] / servidor_dados["Intervalo"].sum()

161.97717621527627

### Uso dos servidores, lembrando que em nosso exemplo $s=2$

In [78]:
servidor_dados["Intervalo"] @ servidor_dados["n"] / servidor_dados["Intervalo"].sum()

1.7416385862493688

### Porcentagem do tempo que não temos ninguem no sistema

In [79]:
servidor_ocioso = servidor_dados[servidor_dados["n"] == 0]
servidor_ocioso

,Tempo,n,n_q,Intervalo
0,0.000000,0.0,0.0,0.543610
2,0.635734,0.0,0.0,1.700585
4,3.043298,0.0,0.0,0.160038


In [80]:
servidor_ocioso["Intervalo"].sum() / servidor_dados["Intervalo"].sum()

0.0050088198317965425